In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Creutzfeldt-Jakob_Disease"
cohort = "GSE62699"

# Input paths
in_trait_dir = "../../input/GEO/Creutzfeldt-Jakob_Disease"
in_cohort_dir = "../../input/GEO/Creutzfeldt-Jakob_Disease/GSE62699"

# Output paths
out_data_file = "../../output/preprocess/Creutzfeldt-Jakob_Disease/GSE62699.csv"
out_gene_data_file = "../../output/preprocess/Creutzfeldt-Jakob_Disease/gene_data/GSE62699.csv"
out_clinical_data_file = "../../output/preprocess/Creutzfeldt-Jakob_Disease/clinical_data/GSE62699.csv"
json_path = "../../output/preprocess/Creutzfeldt-Jakob_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Integrating mRNA and miRNA Co-Expression Networks with eQTLs in the Nucleus Accumbens of Human Chronic Alcoholics"
!Series_summary	"Alcohol consumption is known to lead to gene expression changes in the brain. After performing gene co-expression network analysis (WGCNA) of genome-wide mRNA and microRNA expressions in the Nucleus Accumbens (NAc) from subjects with alcohol dependence (AD) and matched controls six mRNA and three miRNA modules significantly correlated with AD after Bonferroni correction (adj. p≤ 0.05) were identified. Cell-type-specific transcriptome analysis revealed two of the mRNA modules to be enriched for neuronal specific marker genes and downregulated in AD, whereas the remaining four were enriched for astrocyte and microglial specific marker genes and were upregulated in AD. Using gene set enrichment analysis, the neuronal specific modules were enriched for genes involved in oxidative phosphorylation, mitochondrial dysfunction

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analysis of dataset characteristics based on the provided information

# 1. Gene Expression Data Availability
# From the background information, we can see that this dataset contains mRNA expression data
# from Affymetrix GeneChip® Human Genome U133A 2.0 and miRNA data - so it's suitable for our analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Trait Information (Creutzfeldt-Jakob Disease)
# Looking at the background information, cases with Creutzfeldt-Jakob disease were explicitly excluded
# "Cases were excluded if they had an infectious disease (i.e. HIV/AIDS, hepatitis B or C, or Creutzfeldt-Jakob disease)"
# This means the dataset doesn't contain our trait of interest
trait_row = None

# Define conversion function for trait even though we won't use it
def convert_trait(value):
    """Convert trait value to binary format: 1 for case, 0 for control."""
    if pd.isna(value):
        return None
    value_lower = str(value).lower()
    if ':' in value_lower:
        value_lower = value_lower.split(':', 1)[1].strip()
    
    # Not applicable as the dataset doesn't contain CJD cases
    return None

# 2.2 Age Information
# Age is not provided in sample characteristics dictionary
age_row = None

def convert_age(value):
    """Convert age value to continuous format."""
    if pd.isna(value):
        return None
    
    if ':' in str(value):
        value = str(value).split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# 2.3 Gender Information
# Gender is not provided in sample characteristics dictionary
gender_row = None

def convert_gender(value):
    """Convert gender value to binary format: 0 for female, 1 for male."""
    if pd.isna(value):
        return None
    
    value_lower = str(value).lower()
    if ':' in value_lower:
        value_lower = value_lower.split(':', 1)[1].strip()
    
    if 'female' in value_lower or 'f' == value_lower:
        return 0
    elif 'male' in value_lower or 'm' == value_lower:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip clinical feature extraction
if trait_row is not None:
    try:
        # Load clinical data
        clinical_data = pd.read_csv(f"{in_cohort_dir}/clinical_data.csv")
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        print("Clinical Features Preview:")
        print(preview_df(clinical_features))
        
        # Save the clinical features to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical features: {e}")
else:
    print("Clinical data extraction skipped: trait_row is None")


A new JSON file was created at: ../../output/preprocess/Creutzfeldt-Jakob_Disease/cohort_info.json
Clinical data extraction skipped: trait_row is None


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 63
Header line: "ID_REF"	"GSM1531652"	"GSM1531653"	"GSM1531654"	"GSM1531655"	"GSM1531656"	"GSM1531657"	"GSM1531658"	"GSM1531659"	"GSM1531660"	"GSM1531661"	"GSM1531662"	"GSM1531663"	"GSM1531664"	"GSM1531665"	"GSM1531666"	"GSM1531667"	"GSM1531668"	"GSM1531669"	"GSM1531670"	"GSM1531671"	"GSM1531672"	"GSM1531673"	"GSM1531674"	"GSM1531675"	"GSM1531676"	"GSM1531677"	"GSM1531678"	"GSM1531679"	"GSM1531680"	"GSM1531681"	"GSM1531682"	"GSM1531683"	"GSM1531684"	"GSM1531685"	"GSM1531686"	"GSM1531687"
First data line: "14q0_st"	7.68542	7.69338	8.05731	8.03301	7.41483	7.87933	7.61217	7.80203	7.81174	8.22454	7.39808	7.76491	7.5232	8.3171	8.28569	7.57309	8.52417	7.61213	7.49593	7.92926	7.77119	7.8849	8.36847	7.97813	7.82017	8.14878	7.97129	8.30401	7.80492	7.72724	7.80544	7.76272	7.97463	7.87084	7.74456	7.95184
Index(['14q0_st', '14qI-1_st', '14qI-1_x_st', '14qI-2_st', '14qI-3_x_st',
       '14qI-4_st', '14qI-4_x_st', '14qI-5_st', '14qI-6_st', '14qI-6_x_st',
       '14qI-7_st',

### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers in this dataset are not human gene symbols but appear to be probe IDs from
# an Affymetrix microarray platform (like HG-U133A or similar). These identifiers need to be mapped
# to standard human gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for probe ID and gene symbol in the annotation
# Looking at the annotation data, we can see:
# - 'ID' column contains probe IDs like '1007_s_at' that match the gene expression data identifiers
# - 'Gene Symbol' column contains gene symbols like 'DDR1 /// MIR4640'

# 2. Create the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First few mapping entries:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# 4. Save the gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping dataframe shape: (27035, 2)
First few mapping entries:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Gene expression data shape after mapping: (330, 36)
First few gene symbols after mapping:
Index(['ACA1', 'ACA10', 'ACA11', 'ACA12', 'ACA13', 'ACA15', 'ACA16', 'ACA17',
       'ACA18', 'ACA19'],
      dtype='object', name='Gene')
Gene expression data saved to ../../output/preprocess/Creutzfeldt-Jakob_Disease/gene_data/GSE62699.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Final validation - Clinical data is not available for the trait of interest
# Since we determined in Step 2 that trait_row is None (trait data isn't available),
# we shouldn't proceed with clinical data extraction and linking

# Create a dataframe with gene expression data for validation purposes
gene_data_transposed = normalized_gene_data.T
gene_data_transposed['placeholder_column'] = 0  # Add placeholder column for validation

# 3. Validate and save cohort information, reflecting that this dataset 
# has gene data but not the required trait data
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False,  # We determined in Step 2 that trait data isn't available
    is_biased=False,  # Set to False as required by the function
    df=gene_data_transposed,  # Use transposed gene data with placeholder
    note="Dataset explicitly excludes Creutzfeldt-Jakob disease cases as stated in the dataset description."
)

print("Dataset validation complete. The dataset contains gene expression data but does not contain the required trait data.")

Normalized gene data shape: (315, 36)
First few genes with their expression values after normalization:
          GSM1531652  GSM1531653  GSM1531654  GSM1531655  GSM1531656  \
Gene                                                                   
CST12P       6.51726     6.89974     6.43600     6.43231     7.05614   
EAF2         5.47411     6.25696     5.78199     6.13728     6.65247   
SCARNA1      5.62942     5.72439     5.55974     5.26454     5.61008   
SCARNA10     5.38216     5.35127     5.43474     4.82742     5.49208   
SCARNA11    13.89248    14.53975    13.66992    12.27193    13.41468   

          GSM1531657  GSM1531658  GSM1531659  GSM1531660  GSM1531661  ...  \
Gene                                                                  ...   
CST12P       6.76970     6.61207     6.70551     6.84795     7.07075  ...   
EAF2         6.10666     5.74773     6.06232     6.15704     5.77854  ...   
SCARNA1      5.62087     5.41368     5.36728     5.59424     5.22225  ...   
SCARNA